In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.LoadupSamples import LoadupSamples

import pandas as pd
import numpy as np
import polars as pl
import datetime
import seaborn as sns
import lightgbm as lgb
import random
import matplotlib.pyplot as plt
import logging
import time
import re

In [2]:
params = {
    "daysAfterPrediction": None,
    "idxAfterPrediction": 5,
    'timesteps': 60,
    'target_option': 'mean',
    
    "LoadupSamples_tree_scaling_standard": True,
    "LoadupSamples_time_scaling_stretch": True,
    "LoadupSamples_time_inc_factor": 10,
}

In [3]:
timegroup = "group_regOHLCV_over5years"
treegroup = "group_debug"

eval_date = datetime.date(year=2025, month=6, day=13)
evaldates = [eval_date - datetime.timedelta(days=i) for i in range(1, 6)]
start_train_date = datetime.date(year=2022, month=1, day=1)

ls = LoadupSamples(
    train_start_date=start_train_date,
    test_dates=evaldates,
    treegroup=treegroup,
    timegroup=timegroup,
    params=params,
)
lstree = LoadupSamples(
    train_start_date=start_train_date,
    test_dates=evaldates,
    treegroup=treegroup,
    timegroup=None,
    params=params,
)
lstime = LoadupSamples(
    train_start_date=start_train_date,
    test_dates=evaldates,
    treegroup=None,
    timegroup=timegroup,
    params=params,
)

In [4]:
ls.load_samples(main_path = "../src/featureAlchemy/bin/")
lstree.load_samples(main_path = "../src/featureAlchemy/bin/")
lstime.load_samples(main_path = "../src/featureAlchemy/bin/")

Test date 2025-06-08 not found in the database. Omitting.
Non-finite/too-large values in train y tree: 18 samples.
Non-finite/too-large values in train y time: 18 samples.
Removing 18 samples from training data.
Test date 2025-06-08 not found in the database. Omitting.
Non-finite/too-large values in train y tree: 19 samples.
Removing 19 samples from training data.
Test date 2025-06-08 not found in the database. Omitting.
Non-finite/too-large values in train y time: 1234 samples.
Removing 1234 samples from training data.


In [5]:
train_Xtree = ls.train_Xtree
train_ytree = ls.train_ytree
train_Xtime = ls.train_Xtime
train_ytime = ls.train_ytime

test_Xtree = ls.test_Xtree
test_ytree = ls.test_ytree
test_Xtime = ls.test_Xtime
test_ytime = ls.test_ytime

treenames = ls.featureTreeNames
timenames = ls.featureTimeNames

meta_pl_train = ls.meta_pl_train
meta_pl_test = ls.meta_pl_test

In [6]:
print(np.mean(train_ytree)) if train_ytree.any() else None
print(np.mean(train_ytime)) if train_ytime.any() else None

1.0021848188790132
0.5092525671886415


In [7]:
print(timenames)
print(train_Xtime.shape)

['FeatureLSTM_Price' 'FeatureLSTM_Volume']
(15606, 60, 2)
